# Proc Fonksiyonu

* Missing değerler ile kategorik değerleri dönüştürme için ayrı ayrı notebooklarda çalışıldı.


* **hepsiBirArada:** Şimdi ise tek bir fonksiyon ile verdiğimiz dataframe içindeki hem missing hem de kategorik değişkenlerin dönüşümü yapılır, veriler modele besleyebileceğimiz hale gelir.

In [19]:
import pandas as pd
import numpy as np

In [20]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [21]:
from IPython.display import display
from sklearn import metrics
import math

In [22]:
# Asıl dataframe i okumak için

path= "data/buldozers/"

In [23]:
#df = pd.read_csv(f'{path}Train.csv') 

# feather format ile kaydettiğimiz dataFrame i okumak için
df = pd.read_feather('tmp/buldozers_1')

In [24]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start,DateElapsed
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,3,2006-11-16,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,3,2004-03-26,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,1,2004-02-26,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,1,2011-05-19,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,2,2009-07-23,...,23,3,204,False,False,False,False,False,False,1248307200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412693,6333344,10000.0,1919201,21435,149,2.0,2005,NaN,0,2012-03-07,...,7,2,67,False,False,False,False,False,False,1331078400
412694,6333345,10500.0,1882122,21436,149,2.0,2005,NaN,0,2012-01-28,...,28,5,28,False,False,False,False,False,False,1327708800
412695,6333347,12500.0,1944213,21435,149,2.0,2005,NaN,0,2012-01-28,...,28,5,28,False,False,False,False,False,False,1327708800
412696,6333348,10000.0,1794518,21435,149,2.0,2006,NaN,0,2012-03-07,...,7,2,67,False,False,False,False,False,False,1331078400


# ÖNEMLİ 

* **Test ve Validation verileri gerçek hayattaki veriler olmalıydı.**
* Bunun için de verileri **satış tarihlerine(saledate) göre sıralamalıyız.**
* Train verisi olarak dataframedeki ilk sırada yer alan yani eski verileri, Test ve Validation için son sıradaki yani güncel verileri alırız.

In [25]:
# dataFrame'i saledate kolonuna göre sıralı getirir. 

df = df.sort_values(by="saledate")
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start,DateElapsed
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,0,1989-01-17,...,17,1,17,False,False,False,False,False,False,600998400
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410879,6302984,16000.0,1915521,5266,149,99.0,2001,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200
412476,6324811,6000.0,1919104,19330,149,99.0,2004,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200
411927,6313029,16000.0,1918416,17244,149,99.0,2004,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200
407124,6266251,55000.0,509560,3357,149,99.0,1993,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200


In [26]:
len(df.columns)

66

# Bölüm 1

Dataframe deki kolonların kategorik olan sütunları nümerik hale getirmeden önce kategorik olduklarını belirtilir.

# Bölüm 2


In [27]:
# missing value transform

# def fix_missing(df, col, name):
    
    #if is_numeric_dtype(col):
        #if pd.isnull(col).sum:       
            #df[name+"_na"] = pd.isnull(col)   
        #df[name] = col.fillna(col.median)   

In [28]:
# categorize transform to numeric 

# def numericalize(df, col, name):
    
    #if  not is_numeric_dtype(col):
        #df[name] = col.cat.codes + 1

In [29]:
# def proc_df(df, y_fld):
    
    # y_fld ın degerlerini y ye ata
    # sonra dataframe den y_fld ı sil, tahmin edeceğimiz çıktı değerleri(y) modele girdi verilemeyeceğinden
    #y = df[y_fld].values
    #df.drop([y_fld], axis=1, inplace=True) 
    
    # n ve c df.items içerisinde iterasyon yapar. 
    # n: dataframe name leri, c: dataframe columnların degerleri
    # fix_missing ve numericalize a bu iterasyon dönüşleri ile besle
    #for n,c in df.items():
        #fix_missing(df, c, n)
        #numericalize(df, c, n)    
    
    #return df,y

# Bölüm 3

Hem Train hem de Valid vb için transform yapılır

# Önemli !

* Train verisinde missing_value için **df_na column(True/False)** var fakat validasyon verisinde yoksa hata oluşur veya tam zıttı train'de yok validasyon'da varsa yine **hata olur**.


* **Bu hatayı almamak için Train'de df_na column'a eklediğim missing_value ları bir dictionary'de(nan_dict) tutacağım. Bir de missing_value yerine koyduğum değerleri(median vb) tutacağım.**


* Çünkü transformasyon için train'deki missing_value olan ve değerini değiştirdiğim verilerin valid'de de yine aynı şekilde olması gerekir.


* Yine transformasyon için valid'deki missing_value ların yerine koyacağım değerler Train'deki missing_value ların yerine koyduğum değerle(örn. Traindeki median) aynı olması gerekir.

In [30]:
def fix_missing(df, col, name, nan_dict, is_train):
    
    if is_train:                                      #Train verisi içerisindeyse
        if is_numeric_dtype(col):                     #veri tipi nümerikse
            if pd.isnull(col).sum():                  #column içerisisinde missing varsa
                df[name + "_na"] = pd.isnull(col)     #dataframe e columnadi_na ekle. column u True False ile doldur
                nan_dict[name] = col.median()         #missing i olan column u nan_dict e ekle. o column un value'su column'un medianı yap.
                df[name] = col.fillna(nan_dict[name]) #missing leri median değerleriyle doldur
                
    else:                                         #proc_df fonk.'daki is_train parametresi False ise(valid içerisindeyse vb.)          
        if is_numeric_dtype(col):                 #veri tipi nümerikse
            if name in nan_dict:                  #veri nan_dict'in içerisindeyse
                df[name + "_na"] = pd.isnull(col) #valid verisine columadi_na column'u ekle. column u True False ile doldur(hepsi False da olabilir, başta missing şartı koymadık)
                df[name] = col.fillna(nan_dict[name]) #missing leri nan_dict deki degerle(median) doldur
                
            else:                                        #veri nan_dict içerisinde değilse
                df[name] = col.fillna(df[name].median()) #missing leri nan_dict deki degerle(median) doldur
            

In [31]:
# We will have codes starting from 0 (for missing)

def numericalize(df, col, name):
    
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1

In [32]:
def proc_df(df, y_fld, nan_dict=None, is_train=True):
    
    df = df.copy()
    y = df[y_fld].values
    
    df.drop(y_fld, axis=1, inplace=True)
    
    if nan_dict is None:
        nan_dict = {}
        
    for n,c in df.items():
        fix_missing(df, c, n, nan_dict, is_train)
        numericalize(df, c, n)
        
    if is_train:  #veri train verisi içerisindeyse
        return df, y, nan_dict
    
    return df, y

# Bölüm 4

Veriler modele beslenecek hale getirilir.

**NOT** 
* Kaggle test verisetini verdi fakat validasyon verisetini ayrıca vermedi. 

* Train verisi içerisinden validasyon verisini alacağız.
* **Validation için en güncel verileri** yani dataframe'in sonundaki verileri seçeriz.

In [33]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start,DateElapsed
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,0,1989-01-17,...,17,1,17,False,False,False,False,False,False,600998400
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,0,1989-01-31,...,31,1,31,True,False,False,False,False,False,602208000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410879,6302984,16000.0,1915521,5266,149,99.0,2001,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200
412476,6324811,6000.0,1919104,19330,149,99.0,2004,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200
411927,6313029,16000.0,1918416,17244,149,99.0,2004,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200
407124,6266251,55000.0,509560,3357,149,99.0,1993,NaN,0,2012-04-28,...,28,5,119,False,False,False,False,False,False,1335571200


In [34]:
def split_train_val(df, n):
    
    # dataframe den ilk n tanesi, son n tanesi döndürülür
    return df[:n].copy(), df[n:].copy()

In [39]:
n_valid = 1200
n_train = len(df) - n_valid

# ilk n tanesi(eski veriler) Train, son n tanesi(yeni veriler) Valid olsun
raw_train, raw_valid = split_train_val(df, n_train)

In [40]:
x_train, y_train, nas = proc_df(df, 'SalePrice')

AttributeError: Can only use .cat accessor with a 'category' dtype

In [37]:
df["SalePrice"]

205615     9500.0
274835    14000.0
141296    50000.0
212552    16000.0
62755     22000.0
           ...   
410879    16000.0
412476     6000.0
411927    16000.0
407124    55000.0
409203    34000.0
Name: SalePrice, Length: 412698, dtype: float64